# Training Models

The central goal of machine learning is to train predictive models that can be used by applications. In Azure Machine Learning,  you can use scripts to train models leveraging common machine learning frameworks like Scikit-Learn, Tensorflow, PyTorch, SparkML, and others. You can run these training scripts as experiments in order to track metrics and outputs - in particular, the trained models.

## Connect to Your Workspace

The first thing you need to do is to connect to your workspace using the Azure ML SDK.

> **Note**: If the authenticated session with your Azure subscription has expired since you completed the previous exercise, you'll be prompted to reauthenticate.

In [ ]:
workspace_name=""
subscription_id=""
resource_group=""

#for this example your training folder should contain your training script as well as training data 
training_folder=""

In [ ]:
import azureml.core
from azureml.core import Workspace

ws = Workspace.get(
    name=workspace_name,
    subscription_id=subscription_id,
    resource_group=resource_group)

print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

## Use an Estimator to Run the Script as an Experiment

We'll use a generic **Estimator** object to run the training experiment. Note that the default environment for this estimator does not include the **scikit-learn** and **lightgbm** package, so you need to explicitly add that to the configuration. The conda environment is built on-demand the first time the estimator is used, and cached for future runs that use the same configuration; so the first run will take a little longer.

In [ ]:
from azureml.train.estimator import Estimator
from azureml.core import Experiment
from azureml.widgets import RunDetails

script_params = {
    '--output_folder': './outputs'
}

# Create an estimator
estimator = Estimator(source_directory=training_folder,
                      entry_script='train.py',
                      compute_target='local',
                      conda_packages=['scikit-learn'],
                      pip_packages=['lightgbm'],
                      script_params=script_params
                      )

# Create an experiment
experiment_name = 'porto-seguro-training'
experiment = Experiment(workspace = ws, name = experiment_name)

# Run the experiment based on the estimator
run = experiment.submit(config=estimator)
RunDetails(run).show()
run.wait_for_completion()

You can also retrieve the metrics and outputs from the **Run** object.

In [ ]:
# Get logged metrics
metrics = run.get_metrics()
for key in metrics.keys():
        print(key, metrics.get(key))
print('\n')
for file in run.get_file_names():
    print(file)

## Register the Trained Model

Note that the outputs of the experiment include the trained model file (**model.pkl**). You can register this model in your Azure Machine Learning workspace, making it possible to track model versions and retrieve them later.

In [ ]:
from azureml.core import Model

# Insert code to Register the model below



                   

# Once you have registered the model above, the code below allows you to list all registered models
for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')